# Intermediate Importing Data in R
In this course, you will take a deeper dive into the wide range of data formats out there. More specifically, you'll learn how to import data from relational databases and how to import and work with data coming from the web. Finally, you'll get hands-on experience with importing data from statistical software packages such as SAS, STATA, and SPSS.


## Importing data from databases (Part 1)
Many companies store their information in relational databases. The R community has also developed R packages to get data from these architectures. You'll learn how to connect to a database and how to retrieve data from it.

### Establish a connection
The first step to import data from a SQL database is creating a connection to it. As Filip explained, you need different packages depending on the database you want to connect to. All of these packages do this in a uniform way, as specified in the DBI package.

dbConnect() creates a connection between your R session and a SQL database. The first argument has to be a DBIdriver object, that specifies how connections are made and how data is mapped between R and the database. Specifically for MySQL databases, you can build such a driver with RMySQL::MySQL().

If the MySQL database is a remote database hosted on a server, you'll also have to specify the following arguments in dbConnect(): dbname, host, port, user and password. Most of these details have already been provided.

In [3]:
# Load the DBI package
# install.packages("RMySQL")
library(DBI)

# Edit dbConnect() call
con <- dbConnect(RMySQL::MySQL(), 
                 dbname = "tweater", 
                 host = "courses.csrrinzqubik.us-east-1.rds.amazonaws.com", 
                 port = 3306,
                 user = "student",
                 password = "datacamp")

### List the database tables
After you've successfully connected to a remote MySQL database, the next step is to see what tables the database contains. You can do this with the dbListTables() function. As you might remember from the video, this function requires the connection object as an input, and outputs a character vector with the table names.

In [4]:
# Build a vector of table names: tables
tables = dbListTables(con)

# Display structure of tables
str(tables)

 chr [1:3] "comments" "tweats" "users"


### Import users
As you might have guessed by now, the database contains data on a more tasty version of Twitter, namely Tweater. Users can post tweats with short recipes for delicious snacks. People can comment on these tweats. There are three tables: users, tweats, and comments that have relations among them. Which ones, you ask? You'll discover in a moment!

Let's start by importing the data on the users into your R session. You do this with the dbReadTable() function. Simply pass it the connection object (con), followed by the name of the table you want to import. The resulting object is a standard R data frame.

In [5]:
# Import the users table from tweater: users

users = dbReadTable(con, "users")

# Print users
print(users)

  id      name     login
1  1 elisabeth  elismith
2  2      mike     mikey
3  3      thea   teatime
4  4    thomas tomatotom
5  5    oliver olivander
6  6      kate  katebenn
7  7    anjali    lianja


### Import all tables
Next to the users, we're also interested in the tweats and comments tables. However, separate dbReadTable() calls for each and every one of the tables in your database would mean a lot of code duplication. Remember about the lapply() function? You can use it again here! A connection is already coded for you, as well as a vector table_names, containing the names of all the tables in the database.

In [6]:
# Get table names
table_names <- dbListTables(con)

# Import all tables
tables <- lapply(table_names, dbReadTable, conn = con)

# Print out tables
print(tables)

[[1]]
     id tweat_id user_id            message
1  1022       87       7              nice!
2  1000       77       7             great!
3  1011       49       5            love it
4  1012       87       1   awesome! thanks!
5  1010       88       6              yuck!
6  1026       77       4      not my thing!
7  1004       49       1  this is fabulous!
8  1030       75       6           so easy!
9  1025       88       2             oh yes
10 1007       49       3           serious?
11 1020       77       1 couldn't be better
12 1014       77       1       saved my day

[[2]]
  id user_id
1 75       3
2 88       4
3 77       6
4 87       5
5 49       1
6 24       7
                                                                 post
1                                       break egg. bake egg. eat egg.
2                           wash strawberries. add ice. blend. enjoy.
3                       2 slices of bread. add cheese. grill. heaven.
4               open and crush avocado. add 

In [7]:
# close connection
dbDisconnect(con)


[1] TRUE

In [8]:
# check disconnection
con

ERROR while rich displaying an object: Error in .local(dbObj, ...): internal error in RS_DBI_getConnection: corrupt connection handle

Traceback:
1. FUN(X[[i]], ...)
2. tryCatch(withCallingHandlers({
 .     if (!mime %in% names(repr::mime2repr)) 
 .         stop("No repr_* for mimetype ", mime, " in repr::mime2repr")
 .     rpr <- repr::mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(is.raw(rpr), rpr)
 . }, error = error_handler), error = outer_handler)
3. tryCatchList(expr, classes, parentenv, handlers)
4. tryCatchOne(expr, names, parentenv, handlers[[1L]])
5. doTryCatch(return(expr), name, parentenv, handler)
6. withCallingHandlers({
 .     if (!mime %in% names(repr::mime2repr)) 
 .         stop("No repr_* for mimetype ", mime, " in repr::mime2repr")
 .     rpr <- repr::mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(is.raw(rpr), rpr)
 . }, error = error_handler)
7. repr::mime2repr[[mime]]